In [1]:
%%HTML
<style>
    body {
        --vscode-font-family: "KoddiUD 온고딕"
    }
</style>

# 웹 크롤링
- 인터넷 상에 존재하는 데이터를 자동으로 수집하는 행위
- 데이터 분석가에게 데이터를 탐색하고 원하는 조건에 맞는 데이터를 직접 수집하고 저장까지 하기 위한 목적으로 사용
1. 웹 페이지 정보 가져오기
  - 파이썬 Requests 라이브러리 사용
2. HTML 소스를 파싱(분석)하여 원하는 정보 얻기
  - 파이썬 BeautifulSoup 라이브러리 사용

## 0. 개념 알기
1. 사용자는 브라우저로 접속하고 싶은 주소(URL) 입력
2. 브라우저가 해당 주소의 서버에게 페이지 구성 정보를 달라고 요청(request)
3. 웹 서버는 구성에 필요한 정보를 코드(html) 형태로 전달(response)
4. 브라우저는 서버가 전달해준 정보(html)을 해석해서 사용자 화면에 보여줌

## 1. 웹 데이터 수집 라이브러리(BeautifulSoup)
- HTML 및 XML에서 데이터를 쉽게 처리하는 파이썬 라이브러리
- HTML은 태그로 이루어져 있고, 수많은 공백과 변화하는 소스들 때문에 오류가 있을 가능성이 높지만 BeautifulSoup를 이용하면 이러한 오류를 잡아서 고친 후 데이터를 전달해줌

In [2]:
# 설치 방법
# pip install beautifulsoup4

# 사용 방법
from bs4 import BeautifulSoup

In [8]:
# BeautifulSoup HTML 코드 작성
# HTML 코드 작성
html =  """
<html>
	<body>
	<h1 id = 'title'>파이썬 라이브러리 활용!</h1>
	<p id = 'body'>오늘의 주제는 웹 데이터 수집</p>
	<p class = 'scraping'>삼성전자 일별 시세 불러오기</p>
	</body>
<html>
"""

### 1.1 BeautifulSoup HTML 파싱
- soup = BeautifulSoup(html, 'html.parser')
  - html을 파이썬에서 읽을 수 있게 파싱(파이썬 객체로 변환)
  - html이라는 변수에 저장한 html 소스코드를 .parser를 붙여 변환
  - parser는 파이썬의 내장 메소드 

In [9]:
soup = BeautifulSoup(html, 'html.parser')

### 1.2 BeautifulSoup 데이터를 텍스트로 반환
- `for text in soup:`
    - `print(text)`
- soup
  - soup의 데이터를 모두 가져와서 텍스트로 반환
- soup.contents 
  - soup의 데이터를 모두 가져와서 리스트로 변환
- soup.stripped_strings
  - 공백도 함께 제거하여 텍스트로 반환

soup

In [10]:
for text in soup:
  print(text)



<html>
<body>
<h1 id="title">파이썬 라이브러리 활용!</h1>
<p id="body">오늘의 주제는 웹 데이터 수집</p>
<p class="scraping">삼성전자 일별 시세 불러오기</p>
</body>
<html>
</html></html>


soup.stripped_strings

In [11]:
for stripped_text in soup.stripped_strings:
  print(stripped_text)

파이썬 라이브러리 활용!
오늘의 주제는 웹 데이터 수집
삼성전자 일별 시세 불러오기


### 1.3 BeautifulSoup Find 함수
- `scraping = soup.find(class_='scraping')`
  - `scraping.string`
- find 함수는 id, class, element 등을 검색하는 기능
  - find
    - 조건에 해당하는 첫번째 정보만 검색
    - 클래스의 이름을 알 경우, `class_` 형태로 사용
  - find_all
    - 조건에 해당하는 모든 정보 검색
  - string
    - 태그 내부의 텍스트만 출력

In [ ]:
# find 함수 활용
title = soup.find(id='title')
print(title)

<h1 id="title">파이썬 라이브러리 활용!</h1>


In [ ]:
# scraping 조건에 해당하는 첫 번째 정보만 검색
scraping = soup.find(class_='scraping')
print(scraping)

<p class="scraping">삼성전자 일별 시세 불러오기</p>


In [14]:
scraping.string

'삼성전자 일별 시세 불러오기'

### 1.4 BeautifulSoup 웹 크롤링 3단계 과정
- Request
  - 웹 페이지의 URL 이용해서 HTML 문서를 요청
- Response
  - 요청한 HTML 문서를 회신
- Parsing
  - 태그 기반으로 파싱(일련의 문자열을 의미있는 단위로 분해)


In [18]:
# 웹 페이지의 링크를 이용해서 HTML 문서를 요청하기 위해 필요한 라이브러리
import requests

#### 1.4.1 **Request**

1) URL 저장

In [15]:
# URL 저장
# stock_url이라는 변수에 URL 저장
stock_url = 'https://finance.naver.com//item/sise_day.naver?code=005930'

2. User-agent 설정
   - user-agent 확인 사이트
     - http://www.useragentstring.com/
   - 웹 크롤링을 진행하면 종종 페이지에서 아무것도 받아오지 못하는 경우가 발생함
     - 대부분의 서버에서 봇을 차단하기 때문
   - 그래서 user-agent를 headers에 저장하면 오류를 해결할 수 있음

In [16]:
# header에 user-agent 값 저장
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}

3. requests.get()
  - 웹 페이지의 URL 이용해서 HTML 문서를 요청
    - `requests.get(stock_url, headers = header)`
    - URL 값을 파라미터 값으로 입력
    - 해당 사이트는 반드시 헤더 정보를 요구하기 때문에 파라미터 값으로 헤더 입력

In [19]:
requests.get(stock_url, headers = headers)

<Response [200]>

#### 1.4.2 **Response**
- 요청한 HTML 문서를 회신
- `response = requests.get(stock_url, headers = headers)`
  - 서버에서 요청을 받아 처리한 후, 요청자에게 응답 줌
  - HTML 코드 형태

In [20]:
# response 변수에 요청한 HTML 문서를 회신하여 저장
response = requests.get(stock_url, headers = headers)

#### 1.4.3 **Parsing**
- 태그 기반으로 파싱(일련의 문자열을 의미있는 단위로 분해)
- `soup = BeautifulSoup(response.text, 'html.parser')`
  - html을 파이썬에서 읽을 수 있게 파싱
  - response.text에 저장한 html 소스코드를 .parser를 붙여 변환
  - parser는 파이썬의 내장 메소드

In [21]:
# soup 변수에 BeautifulSoup의 객체 생성
# HTML 코드를 파이썬에서 읽을 수 있도록 파싱
soup = BeautifulSoup(response.text, 'html.parser')

### 1.5 BeautifulSoup 반복문으로 일별 종가 구현
1. 200일 일별 종가 정보는 1 page 당 10일의 일별 종가 정보가 담겨 있어서 20page 필요
2. 일별 종가 담긴 URL과 Header 정보로 requests.get 함수 구현
3. 요청한 HTML 문서를 회신하여 response 변수에 저장
4. BeautifulSoup 함수로 HTML을 읽을 수 있도록 soup 변수에 저장
5. page 개수만큼 20번 반복
   1. 'tr' 태그 조건에 해당하는 모든 정보 검색하여 parsing_list 변수에 저장
6. 한 페이지 당 10일의 일별 종가 정보가 담겨있어서 10번 반복
   1. 'td' 태그의 align가 'center'인 값들 중에서 0번째 조건에 해당하는 정보를 검색하여 출력
   2. 'td' 태그의 class가 'num'인 값들 중 0번째 조건에 해당하는 정보 검색하여 출력

In [22]:
# 200일 동안의 일별 종가 정보 가져오는 반복문(1페이지 당 10일 정보 담겨있음)
for page in range(1, 21):
  print (str(page))

  # url + page 번호 합치기
  stock_url = 'http://finance.naver.com/item/sise_day.nhn?code=005930' +'&page='+ str(page)

  # header 정보
  headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}

  # request : 웹 페이지의 URL, header 이용해서 HTML 문서 요청
  # response : 요청한 HTML 문서 회신
  response = requests.get(stock_url, headers = headers)

  # parsing : HTML을 읽을 수 있도록 파싱
  # soup 변수에 BeautifulSoup의 객체 생성
  soup = BeautifulSoup(response.text, 'html.parser')

  # "tr" 태그 조건에 해당하는 모든 정보 검색
  parsing_list = soup.find_all("tr")

  # None 값은 걸러주기 위한 변수 생성
  isCheckNone = None

  # 페이지당 일별 종가 출력하기 위한 반복문 <들여쓰기 주의>
  for i in range(1, len(parsing_list)):
    # None 값은 걸러주기 위한 조건문 <들여쓰기 주의>
    # .span()는 매치된 문자열의 (시작, 끝)에 해당하는 튜플을 돌려주는 함수
    if(parsing_list[i].span != isCheckNone):
      # parsing_list[i] : i번째 parsing_list, i 번째 "tr" 태그 값
      # .find_all("td", align="center")[0].text : "td" 태그의 align가 "center"인 값들 중 0번째 값
      # .find_all("td", class_="num")[0].text : "td" 태그의 class가 "num"인 값들 중 0번째 값
      print(parsing_list[i].find_all("td", align="center")[0].text,
            parsing_list[i].find_all("td", class_="num")[0].text)

1
2024.12.06 54,100
2024.12.05 53,700
2024.12.04 53,100
2024.12.03 53,600
2024.12.02 53,600
2024.11.29 54,200
2024.11.28 55,500
2024.11.27 56,300
2024.11.26 58,300
2024.11.25 57,900
2
2024.11.22 56,000
2024.11.21 56,400
2024.11.20 55,300
2024.11.19 56,300
2024.11.18 56,700
2024.11.15 53,500
2024.11.14 49,900
2024.11.13 50,600
2024.11.12 53,000
2024.11.11 55,000
3
2024.11.08 57,000
2024.11.07 57,500
2024.11.06 57,300
2024.11.05 57,600
2024.11.04 58,700
2024.11.01 58,300
2024.10.31 59,200
2024.10.30 59,100
2024.10.29 59,600
2024.10.28 58,100
4
2024.10.25 55,900
2024.10.24 56,600
2024.10.23 59,100
2024.10.22 57,700
2024.10.21 59,000
2024.10.18 59,200
2024.10.17 59,700
2024.10.16 59,500
2024.10.15 61,000
2024.10.14 60,800
5
2024.10.11 59,300
2024.10.10 58,900
2024.10.08 60,300
2024.10.07 61,000
2024.10.04 60,600
2024.10.02 61,300
2024.09.30 61,500
2024.09.27 64,200
2024.09.26 64,700
2024.09.25 62,200
6
2024.09.24 63,200
2024.09.23 62,600
2024.09.20 63,000
2024.09.19 63,100
2024.09.13 64,40

### 1.6 Pandas 일별 시세 테이블 구현
1) Pandas 라이브러리와 Requests 라이브러리 이용
2) 200일 일별 종가 정보는 1 Page 당 10일의 일별 종가 정보 담겨있어서 20 Page 필요
3) 일별 종가 담긴 URL과 Header 정보로 requests.get 함수 구현
4) pandas.read_html 함수를 통해 HTML 불러와서 파싱
5) concat 함수를 이용하여 dataframe 끝에 추가하고 싶은 요소를 추가하여 dataframe 리턴
6) dropna 함수를 통해 결측 값 제거


In [23]:
# 네이버 금융 일별 시세 테이블 불러오기
# pandas 라이브러리와 requests 라이브러리 이용
import pandas as pd
import requests


# 빈 데이터프레임을 생성하여 이후 각 페이지에서 가져온 데이터를 추가
stock_data = pd.DataFrame()

# code = 회사 코드, page = 일별 시세 테이블의 페이지 수 (200 행의 데이터 불러오려면 20 페이지 입력)
for page in range(1, 21): # ➊ 페이지 순회
   stock_url = 'http://finance.naver.com/item/sise_day.nhn?code=005930' +'&page='+ str(page)

   #  header 정보
   headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}

   # request : 웹 페이지의 URL, header 이용해서 HTML 문서 요청
   # response : 요청한 HTML 문서 회신
   response = requests.get(stock_url, headers = headers)


   # ➋ response.text로 응답을 주면 HTML 코드이기 때문에 read_html로 불러오기
   df_page = pd.read_html(response.text, header=0)[0]


   # ➌ concat() : 여러 DataFrame을 하나로 결합할 때 사용
   # 주의!! : pandas 2.0.0 버전 이후부터 'append()' Method가 완전히 제거되었기 때문에 더 이상 작동하지 않습니다. concat() 사용(설명 아래 참고)
   stock_data = pd.concat([stock_data, df_page], ignore_index=True)


# ➍ 결측값 행 제거(for문 외부에 위치하도록 들여쓰기 주의)
stock_data = stock_data.dropna()
stock_data

/var/folders/yl/lyd4cgp16bj2jd049cmb64n00000gn/T/ipykernel_32162/4231495009.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_page = pd.read_html(response.text, header=0)[0]
/var/folders/yl/lyd4cgp16bj2jd049cmb64n00000gn/T/ipykernel_32162/4231495009.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_page = pd.read_html(response.text, header=0)[0]
/var/folders/yl/lyd4cgp16bj2jd049cmb64n00000gn/T/ipykernel_32162/4231495009.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_page = pd.read_html(response.text, header=0)[0]
/var/folders/yl/lyd4cgp16bj2jd049cmb64n00000gn/T/ipykernel_32162/4231495009.py:23: Fu

,날짜,종가,전일비,시가,고가,저가,거래량
1,2024.12.06,54100.0,상승 400,53900.0,54400.0,52700.0,22805072.0
2,2024.12.05,53700.0,상승 600,53200.0,54400.0,53200.0,23588277.0
3,2024.12.04,53100.0,하락 500,52000.0,53400.0,52000.0,29004766.0
4,2024.12.03,53600.0,보합0,53100.0,54400.0,53100.0,23374603.0
5,2024.12.02,53600.0,하락 600,54300.0,54400.0,53100.0,22044867.0
...,...,...,...,...,...,...,...
294,2024.02.20,73300.0,하락 500,73700.0,73700.0,72800.0,14681477.0
295,2024.02.19,73800.0,"상승 1,000",72800.0,73900.0,72800.0,12726404.0
296,2024.02.16,72800.0,하락 200,73300.0,73400.0,72500.0,13444781.0
297,2024.02.15,73000.0,"하락 1,000",74200.0,74400.0,73000.0,14120600.0
